### 소규모 데이터셋으로 CNN학습하기
- 2,000장 강아지, 2,000장 고양이 사진 = 총 4,000장 데이터
- validation 1,000장, test 1,000장

##### 1. data augmentation 
    - 2,000장의 training set으로 기본 성능을 만든다. *이슈 포인트 :  overfitting
    - overfitting을 줄이기 위해 data augmentation 실시 <br>
    
##### 2. pre-training model 활용
    - pre-trained된 network로 특성 추출
    - pre-trained된 network를 fine-tuning

In [18]:
# 데이터 다운
# 25000 --> 4000 소규모 데이터 셋으로 만든다.
# train 데이터를 train, validation, test 셋으로 나누자

import shutil, os

original_dataset_dir = "./datasets/dogs-vs-cats/train/"

base_dir = "./datasets/dogs-vs-cats_small"
os.mkdir(base_dir)

# 경로 및 디렉토리 생성
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 훈련용 강아지/고양이 데이터셋 디렉토리 생성
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# 검증용 강아지/고양이 데이터셋 디렉토리 생성
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# 테스트용 강아지/고양이 데이터셋 디렉토리 생성
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)
 

In [20]:
# 이미지를 각 디렉토리에 집어넣자
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)] # 처음 1000장 강아지 이미지를 train_dogs_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, cp_src)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1000,1500)] # 다음 500장 강아지 이미지를 validation_dogs_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, cp_src)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1500,2000)] # 다음 500장 강아지 이미지를 test_dogs_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, cp_src)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)] # 처음 1000장 고양이 이미지를 train_cats_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, cp_src)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1000,1500)] # 다음 500장 고양이 이미지를 validation_cats_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, cp_src)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1500,2000)] # 다음 500장 고양이 이미지를 test_cats_dir로 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    cp_src = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, cp_src)  

In [21]:
# 잘 들어갔는지 확인해보자
print("훈련용 강아지 이미지 총 갯수 : ", len(os.listdir(train_dogs_dir)))
print("훈련용 고양이 이미지 총 갯수 : ", len(os.listdir(train_cats_dir)))

print("검증용 강아지 이미지 총 갯수 : ", len(os.listdir(validation_dogs_dir)))
print("검증용 고양이 이미지 총 갯수 : ", len(os.listdir(validation_cats_dir)))

print("테스트용 강아지 이미지 총 갯수 : ", len(os.listdir(test_dogs_dir)))
print("테스트용 고양이 이미지 총 갯수 : ", len(os.listdir(test_cats_dir)))

훈련용 강아지 이미지 총 갯수 :  1000
훈련용 고양이 이미지 총 갯수 :  1000
검증용 강아지 이미지 총 갯수 :  500
검증용 고양이 이미지 총 갯수 :  500
테스트용 강아지 이미지 총 갯수 :  500
테스트용 고양이 이미지 총 갯수 :  500


In [28]:
# 네트워크 구성
from keras import models
from keras.layers import *

model = models.Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(150,150,3))) # output shape은 148x148x32
model.add(MaxPooling2D(pool_size=(2,2))) # output shape은 74x74x32
model.add(Conv2D(64,kernel_size=(3,3), activation='relu')) # output shape은 72x72x64
model.add(MaxPooling2D(pool_size=(2,2))) # output shape은 36x36x64
model.add(Conv2D(128,kernel_size=(3,3), activation='relu')) # output shape은 34x34x128
model.add(MaxPooling2D(pool_size=(2,2))) # output shape은 17x17x128
model.add(Conv2D(128,kernel_size=(3,3), activation='relu')) # output shape은 15x15x128
model.add(MaxPooling2D(pool_size=(2,2))) # output shape은 7x7x128 , 7.5가 아니라 7이 되는구나...
model.add(Flatten()) # 7*7*128 = 6272개 유닛
model.add(Dense(units=512, activation='relu')) # output units = 512
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [30]:
from keras import optimizers

model.compile(optimizer=optimizers.rmsprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

In [33]:
# 데이터 전처리 with 제너레이터
# 0~255 --> 0~1 사잇값으로 조정
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [34]:
# 아래 flow_from_directory는 target directory안에 클래스별 디렉토리가 있어야한다.
# 이미지 크기는 150x150로 resize
# class mode에서 다중분류라면 'categorical'(2차원 one-hot encoding) 또는 'sparse'(1차원 정수 레이블)
train_generator = train_datagen.flow_from_directory(directory=train_dir, target_size=(150,150), batch_size=20, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(directory=validation_dir, target_size=(150,150), batch_size=20, class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [35]:
# 샘플 제너레이터 출력
# 무한정 배치를 만들어주기때문에 break를 걸어줘야한다.

for data_batch, labels_batch in train_generator:
    print("Batch data size : ", data_batch.shape)
    print("Batch labels size : ", labels_batch.shape)
    break

Batch data size :  (20, 150, 150, 3)
Batch labels size :  (20,)


In [36]:
# fit_generator사용해보자
hist = model.fit_generator(train_generator,
                           steps_per_epoch=100,
                           epochs=30,
                           validation_data=validation_generator,
                           validation_steps=50) # epoch당 50개의 validation batch뽑아서 평가하는듯

Epoch 1/30
  5/100 [>.............................] - ETA: 2:23 - loss: 0.7144 - acc: 0.5700

KeyboardInterrupt: 